<h2> Importing modules </h2>

In [1]:
import h5py
import scipy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as color

from scipy.stats import gaussian_kde

# GroEL - 1968
# Ferritin - 1933
# Rubisco - 1928
# MS2 - 1958

<h2> Opening file </h2>
There are several (boolean) flags that control some options. Function is described in comment. 

In [3]:
printVals = True # modify this - show all hit intensities
doAdditional = True # modify this - additional filtering

f_name = f'/home/tejvarmay/scattering_data/data/newdata/data00444.cxi' # modify this - hits file name 

#################################
#### DON'T MODIFY BEYOND HERE ###
#################################

# Initialisation of variables/arrays
hits = 0
frames = 0 
hit_intensity = 0
hit_intensities = np.empty(shape = (0)) 
hit_x = np.empty(shape = (0)) # array for storing x-location of hit 
hit_y = np.empty(shape = (0)) # array for storing y-location of hit 
hit_frame = np.empty(shape = (0), dtype = int) # array for storing frame number of hits

f = h5py.File(f_name, 'r') # opening file 

frames += f['/5_detect/n'].shape[0] 
hits += np.sum(f['/5_detect/n'])
hits_per_frame = np.array(f["/5_detect/n"])

print('Number of frames:', frames)
print('Number of raw hits:', hits)

for i in range(0, f['/5_detect/x'].shape[0]): # frame counter 
    for j in range(0, f['/5_detect/n'][i]): # particle counter 
        if f['/6_analyse/peak_sum'][i, j] > 0: # ignore hits with intensity lower than zero 
            hit_x = np.append(hit_x, f['/5_detect/x'][i, j])
            hit_y = np.append(hit_y, f['/5_detect/y'][i, j]) 
            hit_frame = np.append(hit_frame, i)
        
a = np.array(f['/6_analyse/peak_sum'])
real_hits = np.sum((a > 0)) 
print('Number of real hits (intensity > 0):', real_hits)

hit_intensities = np.append(hit_intensities, a[a > 0]) 

sat_pix = np.array(f["/1_raw/saturated_n_pixels"])
success_pix = np.array(f["/1_raw/success"])
    
num_saturated_pixels = np.sum(sat_pix[sat_pix != 0])
num_saturated_frames = np.sum(success_pix == 0)
num_nonsaturated_frames = np.sum(success_pix == 1)

fraction_saturated = num_saturated_frames / num_nonsaturated_frames

print("Number of saturated pixels: %s" % (num_saturated_pixels))
print("Number of saturated frames: %s/%s" % (num_saturated_frames, frames))
print("Number of non-saturated frames: %s/%s" % (num_nonsaturated_frames,frames))
print("Percentage of saturated frames: %.4s%%" % (fraction_saturated*100))

KeyError: 'Unable to open object (component not found)'

<h2> Outlier removal </h2>
In this section, the processed hits are corrected by discarding obvious outliers. At present this is done using the IQR. Other methods might be used in the future as well. Some statistics are printed before and after outlier removal. It is also possible to print out all the hits sorted by intensity. Different outlier removal methods have to be considered, especially when writing a publication. 

In [ ]:
av_intensity = np.mean(hit_intensities)
med_intensity = np.median(hit_intensities)
std_intensity = np.std(hit_intensities)
min_intensity = np.min(hit_intensities)
max_intensity = np.max(hit_intensities)
mad_intensity = scipy.stats.median_abs_deviation(hit_intensities)

print("------------------------------------------------------")
print("Intensity statistics before processing:")
print("------------------------------------------------------")
print("%s hits before removal." % (real_hits))
print('Average intensity:', av_intensity)
print('Median intensity:', med_intensity)
print("MAD intensity: %s." % (mad_intensity))
print('Standard deviation intensity:', std_intensity)
print('Minimum intensity:', min_intensity)
print('Maximum intensity:', max_intensity)
if printVals:
    with np.printoptions(threshold = np.inf):
        print(np.sort(hit_intensities))
print("------------------------------------------------------")
print() 

### Removing outlier(s) from particle intensities and subsequent calculating sixth-root intensities 
# Calculating IQR
percentile_25 = np.percentile(hit_intensities, q = 25)
percentile_75 = np.percentile(hit_intensities, q = 75)
q1, q3 = percentile_25, percentile_75
iqr = q3 - q1 

iqr_fac = 1.5 
iqr_low = q1 - iqr_fac * iqr 
iqr_high = q3 + iqr_fac * iqr

# Calculating MAD
mad_fac = 4
mad_low = med_intensity - mad_fac * mad_intensity
mad_high = med_intensity + mad_fac * mad_intensity

hits_corrected, hitx, hity, hitframe = [], [], [], []
hits_remain, hits_del = 0, 0
for h in range(len(hit_intensities)): 
    hit_h = hit_intensities[h] 
    hx, hy, hf = hit_x[h], hit_y[h], hit_frame[h]
    if hit_h >= iqr_low and hit_h <= iqr_high: 
        hitx.append(hx) 
        hity.append(hy) 
        hitframe.append(hf)
        hits_corrected.append(hit_h) 
        hits_remain += 1
    else:
        hits_del += 1

hits_corrected = np.array(hits_corrected)
hitx = np.array(hitx) 
hity = np.array(hity) 
hitframe = np.array(hitframe)

print("------------------------------------------------------")
print("Intensity statistics after outlier removal:")
print("------------------------------------------------------")
print("%s hits after removal." % (hits_remain))
print("%s hits removed." % (hits_del)) 
print('average intensity:', np.mean(hits_corrected)) 
print('median intensity:', np.median(hits_corrected)) 
print("MAD intensity: %s." % (scipy.stats.median_abs_deviation(hits_corrected)))
print('standard deviation intensity:', np.std(hits_corrected)) 
print('minimum intensity:', np.min(hits_corrected)) 
print('maximum intensity:', np.max(hits_corrected)) 
if printVals:
    with np.printoptions(threshold = np.inf):
        print(np.sort(hits_corrected))
print("------------------------------------------------------")
print()

# Additional intensity range filtering
if doAdditional:
    c_range_1, c_range_2 = 100, 7000 # change these two values to define custom intensity range
    
    filt = (hits_corrected >= c_range_1) & (hits_corrected <= c_range_2)
    hits_before_additional = hits_corrected.shape[0]
    hits_after_additional = filt.sum()
    
    hits_corrected = hits_corrected[filt]
    hitx, hity = hitx[filt], hity[filt]
        
    print("------------------------------------------------------")
    print("Intensity statistics after additional filtering:")
    print("------------------------------------------------------")
    print("%s hits after removal." % (hits_after_additional))
    print("%s hits removed additionally." % (hits_before_additional-hits_after_additional))
    print('average intensity:', np.mean(hits_corrected)) 
    print('median intensity:', np.median(hits_corrected)) 
    print("MAD intensity: %s." % (scipy.stats.median_abs_deviation(hits_corrected)))
    print('standard deviation intensity:', np.std(hits_corrected)) 
    print('minimum intensity:', np.min(hits_corrected)) 
    print('maximum intensity:', np.max(hits_corrected)) 
    if printVals:
        with np.printoptions(threshold = np.inf):
            print(np.sort(hits_corrected))
    print("------------------------------------------------------")
    print()

<h2> Sixth-root intensity </h2>
After outlier removal we can print out some statistics about the sixth-root intensities. To compare we also show the statistics before outlier removal. If everything is done properly, the standard deviation of the hits shouldn't be higher than the average of the hits. The values below take into account the outlier filtering or the custom intensity range filtering. 

In [ ]:
printSixthRootVals = True # change this to display sorted hit sixth-root intensities
print("------------------------------------------------------")
print("Sixth root intensity statistics before processing:")
print("------------------------------------------------------")
print('average intensity:', np.mean(hit_intensities ** (1/6))) 
print('median intensity:', np.median(hit_intensities ** (1/6))) 
print("MAD intensity: %s." % (scipy.stats.median_abs_deviation(hit_intensities ** (1/6))))
print('standard deviation intensity:', np.std(hit_intensities ** (1/6))) 
print('minimum intensity:', np.min(hit_intensities ** (1/6))) 
print('maximum intensity:', np.max(hit_intensities ** (1/6))) 

if printSixthRootVals: 
    with np.printoptions(threshold = np.inf):
        print(np.sort(hit_intensities ** (1/6)))
print("------------------------------------------------------")
print() 

hits_sixth_root = hits_corrected ** (1/6) 
print("------------------------------------------------------")
print("Sixth root intensity statistics after processing:")
print("------------------------------------------------------")
print('average intensity:', np.mean(hits_sixth_root)) 
print('median intensity:', np.median(hits_sixth_root)) 
print("MAD intensity: %s." % (scipy.stats.median_abs_deviation(hits_sixth_root)))
print('standard deviation intensity:', np.std(hits_sixth_root)) 
print('minimum intensity:', np.min(hits_sixth_root)) 
print('maximum intensity:', np.max(hits_sixth_root)) 

if printSixthRootVals: 
    with np.printoptions(threshold = np.inf):
        print(np.sort(hits_sixth_root))
print("------------------------------------------------------")
print()

<h2> Plotting of results </h2>

In [ ]:
xmin = hitx.min()
xmax = hitx.max()
ymin = hity.min()
ymax = hity.max()
xy = np.vstack([hitx,hity])

z_kde = gaussian_kde(xy)
max_kde = z_kde(xy).max()

xgrid, ygrid = np.mgrid[xmin:xmax:500j, ymin:ymax:500j]
xyg = np.vstack([xgrid.ravel(), ygrid.ravel()])
z_kde_grid = np.reshape(z_kde(xyg).T, xgrid.shape)
       
c = 'viridis' 
num_bins = 50 # change this to allow for finer histograms - only if enough hits detected

xpmin = xmin - 10
ypmin = ymin - 10

xpmax = xmax + 10 
ypmax = ymax + 10 

text_hits = str(hits_remain) + " hits" 

In [ ]:
fig, ax = plt.subplots(4 , 2, figsize = (20, 20), dpi = 300) 

s1 = ax[0,0].scatter(hitx , hity , c = hits_corrected, s = 5, 
                     cmap = c, alpha = 0.7, vmin = np.min(hits_corrected), vmax = np.mean(hits_corrected) + 3 * np.std(hits_corrected)) 
ax[0,0].set_ylim(ypmin, ypmax) 
ax[0,0].set_xlim(xpmin, xpmax)
ax[0,0].set_xlabel('x[pix]', weight='bold', fontsize=8) 
ax[0,0].set_ylabel('y[pix]', weight='bold', fontsize=8)
ax[0,0].set_title('Scatter plot of hits colored by intensity') 
fig.colorbar(s1, ax = ax[0,0]) 

s2 = ax[0,1].scatter(hitx , hity , c = hits_sixth_root, s = 5, cmap = c, alpha = 0.7 , 
                     vmin = np.min(hits_sixth_root), vmax = np.mean(hits_sixth_root) + 3 * np.std(hits_sixth_root)) 
ax[0,1].set_ylim(ypmin, ypmax) 
ax[0,1].set_xlim(xpmin, xpmax)
ax[0,1].set_xlabel('x[pix]', weight='bold', fontsize=8) 
ax[0,1].set_ylabel('y[pix]', weight='bold', fontsize=8) 
ax[0,1].set_title('Scatter plot of hits colored by sixth-root intensity') 
fig.colorbar(s2, ax = ax[0,1])

s3 = ax[1,0].hist2d(hitx, hity, bins = 150, weights = hits_corrected, vmin = 0, vmax = np.max(hits_corrected)) 
ax[1,0].set_xlabel('x[pix]', weight='bold', fontsize=8)
ax[1,0].set_ylabel('y[pix]', weight='bold', fontsize=8) 
ax[1,0].set_title('2D histogram of hits weighted by intensity') 

fig.colorbar(s3[3], ax = ax[1,0]) 

s4 = ax[1,1].hist2d(hitx, hity, bins = 150, weights = hits_sixth_root, vmin = 0, vmax = np.max(hits_sixth_root)) 
ax[1,1].set_xlabel('x[pix]', weight='bold', fontsize=8) 
ax[1,1].set_ylabel('y[pix]', weight='bold', fontsize=8)
ax[1,1].set_title('2D histogram of hits weighted by sixth-root intensity') 
fig.colorbar(s4[3], ax = ax[1,1]) 

ax[2,0].hist(hits_corrected, bins = num_bins)
ax[2,0].set_title('Hit intensities') 
ax[2,0].set_xlim(np.min(hits_corrected), np.max(hits_corrected)) 
ax[2,0].set_xlabel('Intensities', weight='bold', fontsize=8) 
ax[2,0].set_ylabel('Counts', weight='bold', fontsize=8)
ax[2,0].set_xlim(0, np.max(hits_corrected))

ax[2,1].hist(hits_sixth_root, bins = num_bins)
ax[2,1].set_xlim(np.min(hits_sixth_root), np.max(hits_sixth_root)) 
ax[2,1].set_title('Hit intensities - sixth root') 
ax[2,1].set_xlabel('Intensities', weight='bold', fontsize=8)
ax[2,1].set_ylabel('Counts', weight='bold', fontsize=8) 

s5 = ax[3,0].scatter(hitx , hity , c = z_kde(xy), s = 5, alpha = 1.0) 
ax[3,0].set_ylim(ypmin, ypmax)
ax[3,0].set_xlim(xpmin, xpmax) 
ax[3,0].set_xlabel('x[pix]', weight='bold', fontsize=8) 
ax[3,0].set_ylabel('y[pix]', weight='bold', fontsize=8) 
ax[3,0].set_title('Scatter plot of hits colored by KDE')
fig.colorbar(s5, ax = ax[3,0]) 

ax[3,1].plot(np.sort(hit_intensities), 'ro', markerfacecolor = "None", markeredgecolor = "red") 
ax[3,1].set_title("Hit intensities") 
ax[3,1].set_yscale("log") 
ax[3,1].plot(np.sort(hits_corrected), 'bx', markerfacecolor = "None", markeredgecolor = "blue") 
ax[3,1].legend(["Before removal", "After removal"]);

# Save figure
particle_size = 20 # change this to the size in nm
particle_run = '1968' # change this to the type of particle
save_name = f'hit_density_{particle_size}_nm_run_{particle_run}' + '.pdf' # save to the root of the notebook directory

plt.savefig(save_name);
f.close() # closing file handle